In [7]:
from torch.utils.data import Dataset
import torch
# make autoreload
from IPython import get_ipython
import numpy as np
ipython = get_ipython()
ipython.magic('load_ext autoreload')

class PolyDataset(Dataset):
    def __init__(self, degree: int, num_points: int, num_functions: int, num_samples: int = 1):
        self.degree = degree # max degree of each sampled polynomial
        self.num_points = num_points # nr of points to sample from each polynomial as the context length
        self.num_functions = num_functions # nr of plolynomials to use, these will be random at each new step
        # self.num_samples = num_samples # nr of samples to generate for each function
        self.x = torch.linspace(0, 1, num_points)

    def __len__(self):
        return self.num_functions * self.num_points

    def __getitem__(self, idx: int ):
        # for beginning since there is a ton of polynomials to sample from we will just sample a random one
        # need to make sure that numerical evals stay within range of of float32
        # sample random polynomial of degree k
        # sample random points from the polynomial

        # y = a_n * x^n + a_n-1 * x^n-1 + ... + a_0

        # sample k roots
        roots = torch.rand(self.degree) 

        # numpy get random polynomial
        y = self._polynomial(roots, self.x)
        return torch.tensor(y).reshape(-1)
    
    def _polynomial(self, roots, x):
        coeff = np.poly(roots)
        return np.polyval(coeff, x)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_150467/3791145972.py:7: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('load_ext autoreload')


In [2]:
# sample n z_1, z_2, z_3, z_4, z_5, z_6
# poly is now (x-z_1)(x-z_2)(x-z_3)(x-z_4)(x-z_5)(x-z_6)

In [8]:
torch.linspace(0, 1, 6)[None,:]

tensor([[0.0000, 0.2000, 0.4000, 0.6000, 0.8000, 1.0000]])

In [9]:
from model.hippo import HiPPO_LegT

model = HiPPO_LegT(10,dt=0.1)

In [10]:
ds = PolyDataset(15, 10, 1)
dl = torch.utils.data.DataLoader(ds, batch_size=3, shuffle=True)

In [11]:
for i, y in enumerate(dl):
    model.forward_batched(y)
    break

torch.Size([3, 10])
torch.Size([3, 10])
torch.Size([10, 10]) torch.Size([10]) torch.Size([10]) torch.Size([1])


RuntimeError: 1D tensors expected, but got 2D and 1D tensors